In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


n_embd=64
dropout=0.0
n_head=4
n_layer=4
eval_iters=200
eval_interval=100
max_iters=5000
device="cuda" if torch.cuda.is_available() else "cpu"
block_size=32 #max context length for prediction
batch_size=16 #independent sequences processed in parallel

In [ ]:
#@ Dataset for training gpt, we are using is tiny shakespares dataset:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-08-05 11:23:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-08-05 11:23:21 (23.8 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f: #utf-8: represents every characters in byte which is neccessary for the computers
  text=f.read()

In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)

In [ ]:
#@ Creating maps:
string_to_int={ch:i for i, ch in enumerate(chars)}
int_to_string={i:ch for i, ch in enumerate(chars)}

#@ Encoder:
encode=lambda s: [string_to_int[ch] for ch in s]

#@decoder:
decode=lambda l: ''.join([int_to_string[i] for i in l])


In [ ]:
#@ Train and Test Split:
data=torch.tensor(encode(text), dtype=torch.long)
n=int(len(data)*0.8)
train_data=data[:n]
val_data=data[n:]

In [ ]:
#@ Data Loading:
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix=torch.randint(len(data)- block_size, (batch_size, ))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y=x.to(device), y.to(device)
  return x, y

In [ ]:
@torch.no_grad()
def get_loss():
  out={}
  model.eval()
  for split in ['train', 'val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y= get_batch(split)
      logits, loss=model(X, Y)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)

  def forward(self, x):
    B, T, C= x.shape
    q=self.query(x) #B, T, C ->B, T, 16
    k=self.key(x) #B,T, C -> B, T, 16
    weights= q @ k.transpose(-2, -1) #B, T,16 @ B, 16, T -> B, T, T
    weights=weights.masked_fill(self.tril[:T, :T]==0, float('inf'))
    weights=F.softmax(weights, dim=-1)
    weights=self.dropout(weights)
    v=self.value(x) #B, T, C -> B, T, 16
    output= weight @ v #B, T, T @ B, T, 16 -> B, T, 16
    return output

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    self.heads= nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dopout(dropout)

  def forward(self, x):
    output= torch.cat([h(x) for h in self.heads], dim=-1) #B, T, 16 -> B, T, num_heads * 16(this gives n_embd)
    output=self.dropout(self.proj(output))
    return output

In [ ]:
class FeedForwardLayer(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.network=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd)
        nn.Dropout(dropout)
    )
  def forward(self, x):
    return self.network(x)

- Pre-Layer Normalization is a kind of normalization techniques which helps to make training process more stable by eradicating vanishing gradients or exploding gradient
- post layer norm, used in transformer architecture where deep neural nets has gradient issue
- GPT, bert uses pre LN

In [ ]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttention(num_heads, head_size)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)
    self.ffwd=FeedForward(n_embd)

  def forward(self, x):
    x=x + self.sa(self.ln1(x))
    x=x+ self.ffwd(self.ln2(x))
    return x

## Bigram  Model:
(prediction based on the previous words)

In [ ]:
class BigramModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embd) #helps model understand tokens in continuous vector space
    self.positional_embedding_table=nn.Embedding(block_size, n_embd)
    self.blocks= nn.Sequential(*[Block(n_head, n_embd) for _ in range(n_layer)])
    self.ln_f=nn.LayerNorm(n_embd)
    self.lm_head=nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T=idx.shape #batch, t=no of tokens in each sequence
    tok_emb=self.token_embedding_table(idx) #B, T, C
    pos_emb=self.positional_embedding_table(torch.arange(T, device=device)) #T, C
    x=tok_emb + pos_emb #B, T, C
    x=self.blocks(x) #B, T, C
    x=self.ln_f(x) #B, T, C
    logits=self.lm_head(x) #B, T, vocab_size

    if targets==None: #targets for training
      loss=None

    else:
      B, T, C= logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      idx_cond=idx[:, -block_size:]
      #for prediction:
      logits, loss=self(idx_cond)
      logits=logits[:, -1, :] #B, C
      #applying softmax to get probabilities:
      probs=F.softmax(logits, dim=-1) #B, C
      idx_next=torch.multinomial(probs, num_samples=1) #it helps to find the highest probability, B, 1
      idx=torch.cat((idx, idx_next), dim=1)
    return idx

In [ ]:
model=BigramModel()
m=model.to(device)

optim=torch.optim.AdamW(model.parameters(), lr=1e-3)

for iter in range(max_iters):
  if iter % eval_interval == 0 or iter==max_iters-1:
    losses=get_loss()
    print(f"Step {iter + 1}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  #sampling a batch of data:
  xb, yb=get_batch('train')

  #evaluating:
  logits, loss=model(xb, yb)
  optim.zero_grad(set_to_none=True)
  loss.backward()
  optim.step()

# generate from model:
context=torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Step 1: train loss nan, val loss nan
Step 101: train loss nan, val loss nan
Step 201: train loss nan, val loss nan
Step 301: train loss nan, val loss nan
Step 401: train loss nan, val loss nan
Step 501: train loss nan, val loss nan
Step 601: train loss nan, val loss nan
Step 701: train loss nan, val loss nan
Step 801: train loss nan, val loss nan
Step 901: train loss nan, val loss nan
Step 1001: train loss nan, val loss nan
Step 1101: train loss nan, val loss nan
Step 1201: train loss nan, val loss nan
Step 1301: train loss nan, val loss nan
Step 1401: train loss nan, val loss nan
Step 1501: train loss nan, val loss nan
Step 1601: train loss nan, val loss nan
Step 1701: train loss nan, val loss nan
Step 1801: train loss nan, val loss nan
Step 1901: train loss nan, val loss nan
Step 2001: train loss nan, val loss nan
Step 2101: train loss nan, val loss nan
Step 2201: train loss nan, val loss nan
Step 2301: train loss nan, val loss nan
Step 2401: train loss nan, val loss nan
Step 2501: t

KeyboardInterrupt: 